## Demo notebook for FOSDEM2019 graph devroom

In [ ]:
/*
   Tested with beakerx-1.3.0 on python-3.6.7
*/
%classpath add jar target/fosdem2019-0.0.1-SNAPSHOT.jar
%classpath add jar gremlinjars/lib

import nl.vtslab.fosdem2019.traversal.AuthorizedTraversalSource
import static org.apache.tinkerpop.gremlin.structure.VertexProperty.Cardinality.set;
import org.apache.tinkerpop.gremlin.tinkergraph.structure.TinkerGraph

src1Authz = "biz;3";
src2Authz = "fb;2";
src3Authz = "fin;3";
src4Authz = "fin;2";
src5Authz = "fin;4";

class DemoGraph {
    static def createGraph(src1Authz, src2Authz, src3Authz, src4Authz, src5Authz) {
        def graph = TinkerGraph.open();
        def g = graph.traversal();
        g.addV("Person").property("name", "p0").property("authz", src1Authz).property(set, "authz", src2Authz).next();
        g.addV("Person").property("name", "p1").property("authz", src3Authz).next();
        g.addV("Person").property("name", "p2").property("authz", src4Authz).next();
        g.addV("Event").property("name", "v1").property("authz", src1Authz).next();
        g.addV("Event").property("name", "v2").property("authz", src4Authz).next();
        g.V().has("name", "v1").as("a").
                V().has("name", "p0").addE("Visits").to("a").property("name", "e01").property("authz", src1Authz).
                V().has("name", "p1").addE("Visits").to("a").property("name", "e11").property("authz", src3Authz).next();
        g.V().has("name", "v2").as("a").
                V().has("name", "p1").addE("Visits").to("a").property("name", "e12").property("authz", src3Authz).
                V().has("name", "p2").addE("Visits").to("a").property("name", "e22").property("authz", src5Authz).next();
        return graph;
    }
}

graph = DemoGraph.createGraph(src1Authz, src2Authz, src3Authz, src4Authz, src5Authz);
g = graph.traversal();
// Every withAuthorization() call needs a new AuthorizedTraversalSource
ga = {x -> graph.traversal(AuthorizedTraversalSource.class)}
"graph and traversal sources available"

<img src="demograph.png" width="400" align="left"/>

## Normal operation

In [ ]:
unrestricted = g.V().values("name").toList()
authorized = ga().withAuthorization(["biz;3"]).V().values("name").toList()      // <===
"unrestricted: " + unrestricted + "\nauthorized: " + authorized

In [ ]:
unrestricted = g.V().has("name", "p1").out().values("name").toList()
authorized = ga().withAuthorization(["fin;2","fin;3"]).V().has("name", "p1").out().values("name").toList()      // <===
"unrestricted: " + unrestricted + "\nauthorized: " + authorized

## Unauthorized use

In [ ]:
try {
    ga().V().toList();      // <===
    fail("Unauthorized query should fail");
} catch (RuntimeException exception) {
    exception.getMessage();
}

In [ ]:
try {
    ga().withAuthorization(["biz;3"]).withAuthorization(["biz;3", "fin;3"]).V().toList();      // <===
    fail("Query with second withAuthorization() call should fail");
} catch (Exception exception) {
    exception.getMessage();
}

## Trying to manipulate the internal userAuthorization variable

In [ ]:
authorized = ga().withAuthorization(["biz;3"]).
    withSideEffect("userAuthorization", ["biz;3", "fin;3"]).V().values("name").toList();      // <===
"" + authorized + " withSideEffect() to set userAuthorization is ignored"
// Calling anything, including withSideEffect(), before withAuthorization() results in an exception

In [ ]:
try {
    ga().withAuthorization(["biz;3"]).V().
        as("x").inject("fin;3").store("userAuthorization").select("x").toList();      // <===
    fail("Faking userAuthorizations using the store() step should fail");
} catch (Exception exception) {
    "Cannot add to unmodifiable list: " + exception.getClass()
}
// Using aggregate() step instead of store() step gives same result

In [ ]:
try {
    authorizations = ["fin;2", "fin;4"];
    result = ga().withAuthorization(["fin;2"]).V().has("name", "p2").map({t ->
        t.sideEffects("userAuthorization", authorizations);
        return t.get();
    }).out().toList();      // <===
    fail("Accessing AuthorizedTraversal.map(Function) should fail");
} catch (Exception exception) {
    exception.getMessage()
}
// Apart from the map() step also the barrier(), branch(), flatMap(), filter(), emit(), sideEffect() and 
// until() steps provide Traverser instances and need blocking

## Trying to access the graph instance

In [ ]:
try {
    ga().withAuthorization(["biz;3"]).V().getGraph().get().traversal().V().toList();
    fail("Accessing DefaultAuthorizedTraversal.getGraph() should fail");
} catch (Exception exception) {
    exception.getMessage()
}

In [ ]:
testClass = Class.forName("DemoGraph");
testObject = testClass.getConstructor().newInstance();
vertices = testObject.createGraph("biz;3", "fb;2", "fin;3", "fin;2", "fin;4").
    traversal().V().values("name").toList();
"Applications using AuthorizedTraversalSource need to set the JVM SecurityManager: " + vertices

## Trying to access the graph via an anonymous traversal

In [ ]:
import nl.vtslab.fosdem2019.traversal.__

result = ga().withAuthorization(["biz;3"]).V().
    map(__.V().fold()).unfold().dedup().values("name").toList()
"Anonymous __.V() inherits userAuthorization from parent: " + result

In [ ]:
import nl.vtslab.fosdem2019.traversal.__
result = ga().withAuthorization(["biz;3"]).V().has("name", "v1").
    map(__.inE().fold()).unfold().dedup().values("name").toList()
"Anonymous __.inE() inherits userAuthorization from parent: " + result